In [1]:
# !pip install --upgrade pandas
# !pip install --upgrade --ignore-installed PyYAML
# !pip install --upgrade pip
# !pip install --upgrade gen3 --user --upgrade
# !pip install cdiserrors
# !pip install --upgrade pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 5.6 MB/s eta 0:00:0000:01
  Using cached PyYAML-6.0-cp38-cp38-macosx_10_9_x86_64.whl (192 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 8.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.6/526.6 kB 10.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     

In [1]:
import pandas as pd
import sys, os, webbrowser
import gen3
import pydicom
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
import os
# from dicom_csv import join_tree
import subprocess
import zipfile




from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query

In [2]:
# # Import some custom Python scripts from personal GitHub repo.
# # Change these directory paths to reflect your local working directory.

# home_dir = "/Users/hanchen" 
# demo_dir = "{}/Downloads/Research Working/Federal Learning".format(home_dir)

# print(demo_dir)
# os.chdir(demo_dir)

# os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O {}/expansion.py".format(demo_dir))
# %run expansion.py


/Users/hanchen/Downloads/Research Working/Federal Learning


--2023-05-17 19:18:07--  https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211548 (207K) [text/plain]
Saving to: ‘/Users/hanchen/Downloads/Research’

     0K .......... .......... .......... .......... .......... 24% 7.19M 0s
    50K .......... .......... .......... .......... .......... 48% 8.42M 0s
   100K .......... .......... .......... .......... .......... 72% 13.9M 0s
   150K .......... .......... .......... .......... .......... 96% 11.7M 0s
   200K ......                                                100%  132K=0.02s

2023-05-17 19:18:07 (9.95 MB/s) - ‘/Users/hanchen/Downloads/Research’ saved [211548/211548]

--2023-05-17 19:18:07--  http://working/Federal
Resolvin

In [3]:
# Initiate instances of the Gen3 SDK Classes using credentials file for authentication.
# Change the directory path in "cred" to reflect the location of your credentials file.

# api = "https://data.midrc.org"
# cred = "/Users/hanchen/Downloads/credentials.json"
# auth = Gen3Auth(api, refresh_file=cred) # authentication class
# sub = Gen3Submission(api, auth) # submission class
# query = Gen3Query(auth) # query class
# exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
# exp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_REDCORAL', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']


['Open-A1',
 'Open-A1_PETAL_REDCORAL',
 'Open-R1',
 'TCIA-COVID-19-AR',
 'TCIA-COVID-19-NY-SBU',
 'TCIA-COVID-19_CT_Images',
 'TCIA-RICORD']

In [10]:
# Reading a CSV file from a file path
csv_file_path = os.getcwd()+"/datasets/MIDRC_Imaging_Studies_table.csv"
print(csv_file_path)
csv_data = pd.read_csv(csv_file_path, sep=',', header=0)
csv_data.head()


/Users/hanchen/Downloads/Research Working/Federal Learning/APPFL/examples/datasets/MIDRC_Imaging_Studies_table.csv


,ethnicity_0,days_from_study_to_neg_covid_test_0,loinc_long_common_name,loinc_method,body_part_examined_0,days_to_study,index_event_0,study_modality_0,project_id,age_at_imaging,...,sex_0,age_at_index_0,submitter_id,data_category_0,study_year_shifted,covid19_positive_0,object_id_0,loinc_system,loinc_code,data_type_0
0,Not Hispanic or Latino,-169.0,XR Chest PA and Lateral,XR,NaN,288,First COVID test,CR,Open-A1,78.0,...,Female,78,2.16.840.1.114274.1818.53163927261114619887594...,CR,True,No,dg.MD1R/efcaa58b-7db9-40e2-a3e3-ae9fd95de4cc,Chest,42272-5,DICOM
1,Not Hispanic or Latino,NaN,XR Unspecified body region Views,XR,NaN,207,First COVID test,CR,Open-A1,NaN,...,Male,4,2.16.840.1.114274.1818.50672715881797185011680...,CR,True,Yes,dg.MD1R/f953fc0a-761b-445b-bdde-d7af43dc2a27,Unspecified,43468-8,DICOM
2,Not Hispanic or Latino,178.0,XR Chest 2 Views,XR,CHEST,73,First COVID test,CR,Open-A1,40.0,...,Female,40,2.16.840.1.114274.1818.48043293212647531021430...,CR,True,Yes,dg.MD1R/2bba7d00-2394-449d-914b-39e5deff64a1,Chest,36643-5,DICOM
3,Not Hispanic or Latino,79.0,XR Chest AP,XR,CHEST,21,NaN,CR,Open-R1,65.0,...,Male,65,1.2.826.0.1.3680043.10.474.419639.376471887228...,CR,True,No,dg.MD1R/0b3674b0-39de-4c0f-9242-77fe0471049d,Chest,36572-6,DICOM
4,Not Hispanic or Latino,434.0,Portable XR Chest AP single view,XR.portable,CHEST,110,First COVID test,CR,Open-R1,86.0,...,Female,86,1.2.826.0.1.3680043.10.474.232451.69881,CR,True,No,dg.MD1R/46c0b418-68d9-4c94-9c14-453ec28523da,Chest,36589-0,DICOM


In [5]:
# object_ids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855']

# output_addr = "/Users/hanchen/Downloads/Research Working/Federal Learning/"


In [11]:
def dcm_to_png(image_path):
    ds = pydicom.dcmread(image_path)    
    new_image = ds.pixel_array.astype(float)
    scaled_image = np.uint8((np.maximum(new_image, 0) / new_image.max()) * 255.0)    
    final_image = Image.fromarray(scaled_image)
    final_image.save(image_path[:-4]+'.png')

In [46]:
def unzip_raw_data(case_id):
    image_path = os.getcwd()+"/datasets/MIDRC_Data/{}/".format(case_id)
    # "/Users/hanchen/Downloads/Research Working/Federal Learning/{}/".format(case_id)

    study_uids = os.listdir(image_path)

    print(image_path)
    print(study_uids)

    for study_uid in study_uids:
        if study_uid.startswith('.'):
            continue;
        if not os.path.isdir(image_path+study_uid):
            continue
        file_names = os.listdir(image_path+study_uid)
        # print(file_names)
        for file_name in file_names:
            if file_name[-3:]!='zip':
                continue;
            folder_name = []
            with zipfile.ZipFile(image_path+study_uid+'/'+file_name, 'r') as zip_file:
                zip_file_names = zip_file.namelist()
                for zip_file_name in zip_file_names:
                    # print(zip_file_name)
                    # Record folder name
                    folder_name = zip_file_name[:zip_file_name.index('/')]

                    # Specify the file name or path within the ZIP archive
                    file_to_fetch = zip_file_name
                    # print('file:',zip_file_name)
                    
                    # Extract the file to the desired location
                    zip_file.extract(file_to_fetch, image_path)
                    # print("File extracted successfully.")
                    
                    # print('FROM:',image_path+zip_file_name)
                    # print('TO:  ',image_path+zip_file_name[zip_file_name.index('/')+1:])
                    os.rename(image_path+zip_file_name, image_path+zip_file_name[zip_file_name.index('/')+1:])
            os.rmdir(image_path+folder_name)
    file_names = os.listdir(image_path)
    for file_name in file_names:
        if file_name[-3:]=='dcm':
            dcm_to_png(image_path+file_name)

In [52]:
for index, row in csv_data.iterrows():
    unzip_raw_data(row[12])
    print(f"unzip {index} successfully");
    # Access row values using column names or indices
    for i in range(len(row)):
        column1_value = row[i]
        print(f"Row {index}, Column {i}: {column1_value}")

/Users/hanchen/Downloads/Research Working/Federal Learning/APPFL/examples/datasets/MIDRC_Data/10008204-LhAJuYzJS0lnu40xcrgrw/
['2.16.840.1.114274.1818.53163927261114619887594836486386442162', '2.16.840.1.114274.1818.46982739162846728953947451112864564638.dcm', '.DS_Store', '2.16.840.1.114274.1818.46982739162846728953947451112864564638.png']
unzip 0 successfully
Row 0, Column 0: Not Hispanic or Latino
Row 0, Column 1: -169.0
Row 0, Column 2: XR Chest PA and Lateral
Row 0, Column 3: XR
Row 0, Column 4: nan
Row 0, Column 5: 288
Row 0, Column 6: First COVID test
Row 0, Column 7: CR
Row 0, Column 8: Open-A1
Row 0, Column 9: 78.0
Row 0, Column 10: 772
Row 0, Column 11: DCM
Row 0, Column 12: 10008204-LhAJuYzJS0lnu40xcrgrw
Row 0, Column 13: 2.16.840.1.114274.1818.53163927261114619887594836486386442162
Row 0, Column 14: White
Row 0, Column 15: CHEST PA & LATERAL (RAD)-CS
Row 0, Column 16: Female
Row 0, Column 17: 78
Row 0, Column 18: 2.16.840.1.114274.1818.53163927261114619887594836486386442162